# Data Processing

In [2]:
import csv
from requests.exceptions import Timeout, ConnectionError
from requests.packages.urllib3.exceptions import ReadTimeoutError
import ssl
from tweepy import TweepError
import time
import re
import pandas as pd
import os

### Data process for papi feature extraction output, change raw output into csv files, with 37 (number of features) columns

In [79]:
dir_home = './ContestData/papi_out/'
year = [
#     '2013', 
#     '2014', 
#     '2019',
#     '2015',
    '2016',
#     '2017',
#     '2018',
#     '2020',
]
header = ['PAPI_TOT_INS', 'PAPI_TOT_CYC', 'PAPI_L2_ICH', 'PAPI_L2_ICA', 'PAPI_L2_ICM', 'PAPI_L2_TCH', 
          'PAPI_L2_TCA', 'PAPI_L2_TCM', 'PAPI_L2_DCH', 'PAPI_L2_DCA', 'PAPI_L2_DCM', 'PAPI_L1_ICR', 
          'PAPI_L1_ICA', 'PAPI_L1_ICM', 'PAPI_L1_TCH', 'PAPI_L1_TCA', 'PAPI_L1_TCM', 'PAPI_L1_DCH', 
          'PAPI_L1_DCA', 'PAPI_L1_DCM', 'PAPI_TLB_DM', 'PAPI_TLB_IM', 'PAPI_TLB_TL', 'PAPI_STL_ICY', 
          'PAPI_HW_INT', 'PAPI_BR_TKN', 'PAPI_BR_MSP', 'PAPI_BR_INS', 'PAPI_VEC_INS', 'PAPI_RES_STL', 
          'PAPI_FML_INS', 'PAPI_FAD_INS', 'PAPI_FDV_INS', 'PAPI_FSQ_INS', 'PAPI_FP_OPS', 'PAPI_SP_OPS', 'PAPI_DP_OPS']
# flag = True
for y in year:
#     if flag == False:
#         break
    dir_name = dir_home + y + '_output'
    for root, _, files in os.walk(dir_name):
        for file in files:
#             if flag == False:
#                 break
            if '.txt' not in file:
                continue
            fin_name = dir_name + '/' + file
            print(fin_name)
            fout_name = dir_name + '/' + file[:-4] + '.csv'
            print(fout_name)
#             flag = False
            mat = [[] for i in range(37)]
            with open(fin_name) as fin:
                with open(fout_name, 'w', newline = '') as fout:
                    reader = csv.reader(fin)
                    cnt = -1
                    for row in reader:
#                         print(row)
                        if 'round of counting' in row[0]:
#                             print(row)
                            cnt += 1
                        else:
                            mat[cnt].append(row[0])
                    exm_cnt = len(mat[0])
                    print(exm_cnt)
#                     print(mat)
                    writer = csv.writer(fout)
                    writer.writerow(header)
                    rows = [[] for i in range(exm_cnt)]
                    for i in range(len(mat)):
                        for j in range(len(mat[0])):
#                             print(i, j)
                            rows[j].append(mat[i][j])
                    writer.writerows(rows)           

./ContestData/papi_out/2020_output/pipeDK.txt
./ContestData/papi_out/2020_output/pipeDK.csv
9
./ContestData/papi_out/2020_output/newfiberDK.txt
./ContestData/papi_out/2020_output/newfiberDK.csv
37
./ContestData/papi_out/2020_output/visualDK.txt
./ContestData/papi_out/2020_output/visualDK.csv
97
./ContestData/papi_out/2020_output/messengerDK.txt
./ContestData/papi_out/2020_output/messengerDK.csv
46
./ContestData/papi_out/2020_output/evolutionDK.txt
./ContestData/papi_out/2020_output/evolutionDK.csv
35


### Merge features (from papi) and labels (from gem5) into a single csv file

In [91]:
# year = '2013'
# padding = 2
# year = '2014'
# padding = 2 # B is 3
# year = '2015'
# padding = 0
year = '2016'
padding = 2
# year = '2017'
# padding = 0
# year = '2018'
# padding = 0
# year = '2019'
# padding = 0
# year = '2020'
# padding = 0

srcdir_papi = './ContestData/papi_out/finals' + year + '-papi/'
srcdir_gem = './ContestData/ACMFinalsSolutions/finals' + year + '-tick/'
flag = True
for root_p, _, files_p in os.walk(srcdir_papi):
    print(root_p)
    print(files_p)
    for file_p in files_p:
#         if flag == False:
#             break
        key = file_p[:-6] #TODO
        print(key)
        for root_g, _, files_g in os.walk(srcdir_gem):
#             print(root_g)
#             print(files_g)
            for file_g in files_g:
                if '.tick' not in file_g:
                    continue
                if key in file_g:
#                     print('find match %s %s' % (file_p, file_g))
                    file_name_papi = srcdir_papi + file_p
                    file_name_gem = srcdir_gem + file_g
                    print('find match %s %s' % (file_name_papi, file_name_gem))
                    flag = False
                    fout_name = srcdir_gem + file_g
                    fout_name = fout_name[:-5] + '.csv'
                    print(fout_name)
                    with open(file_name_papi) as finpapi:
                        with open(file_name_gem) as fingem:
                            with open(fout_name, 'w', newline='') as fout:
                                reader1 = csv.reader(finpapi)
                                reader2 = csv.reader(fingem)
                                writer = csv.writer(fout)
                                header = next(reader1)
                                header.append('label')
    #                             print(header)
                                writer.writerow(header)
                                if padding == 2:
                                    _ = next(reader1)
                                    _ = next(reader1)
                                rows1 = list(reader1)
                                rows2 = list(reader2)
                                print(len(rows1), len(rows2))
                                assert(len(rows1) == len(rows2) 
#                                        or len(rows1) == len(rows2)-1
                                      )
                                for i in range(len(rows1)):
                                    if len(rows2[i]) == 0:
                                        continue
                                    else:
                                        comb_row = rows1[i]
                                        comb_row.append(rows2[i][0])
#                                         print(comb_row)
                                        writer.writerow(comb_row)
                                    

./ContestData/papi_out/finals2016-papi/
['foreveryoungDK.csv', '.DS_Store', 'ceilingDK.csv', 'oilDK.csv', 'stringDK.csv', 'swapspaceDK.csv', 'marsDK.csv', 'puzzleDK.csv', 'branchDK.csv', 'clockDK.csv', 'balanceddietDK.csv']
foreveryoung
find match ./ContestData/papi_out/finals2016-papi/foreveryoungDK.csv ./ContestData/ACMFinalsSolutions/finals2016-tick/E-foreveryoung.tick
./ContestData/ACMFinalsSolutions/finals2016-tick/E-foreveryoung.csv
58 58
.DS
ceiling
find match ./ContestData/papi_out/finals2016-papi/ceilingDK.csv ./ContestData/ACMFinalsSolutions/finals2016-tick/C-ceiling.tick
./ContestData/ACMFinalsSolutions/finals2016-tick/C-ceiling.csv
38 38
oil
find match ./ContestData/papi_out/finals2016-papi/oilDK.csv ./ContestData/ACMFinalsSolutions/finals2016-tick/G-oil.tick
./ContestData/ACMFinalsSolutions/finals2016-tick/G-oil.csv
36 36
string
find match ./ContestData/papi_out/finals2016-papi/stringDK.csv ./ContestData/ACMFinalsSolutions/finals2016-tick/L-string.tick
./ContestData/ACMFin

### Merge csv files for multiple problems in a single year

In [92]:
dir_home = './ContestData/'
year = [
#     '2013', 
#     '2014', 
#     '2015',
    '2016',
#     '2017',
#     '2018',
#     '2019',
#     '2020',
]
for y in year:
    first = True
    dir_name = dir_home + 'train' + y + '/'
    fout_name = dir_home + 'train' + y + '_all.csv'
    print(fout_name)
    with open(fout_name, 'w', newline='') as fout:
        writer = csv.writer(fout)
        for root, _, files in os.walk(dir_name):
            print(root)
            print(files)
            for file in files:
                file_name = dir_name + file
                print(file_name)
                with open(file_name) as fin:
                    reader = csv.reader(fin)
                    header = next(reader)
                    if first:
                        writer.writerow(header)
                        first = False
                    for row in reader:
                        writer.writerow(row)

./ContestData/train2016_all.csv


### Generate whole dataset by merging csv files from each year (2013-2019)

In [85]:
dir_home = './ContestData/'
year = [
    '2013', 
    '2014', 
    '2015',
#     '2016',
    '2017',
    '2018',
    '2019',
#     '2020',
]
first = True
fout_name = dir_home + 'train_all.csv'
with open(fout_name, 'w', newline='') as fout:
    writer = csv.writer(fout)
    for y in year:
        fin_name = dir_home + 'train' + y + '_all.csv'
        print(fin_name)
        with open(fin_name) as fin:
            reader = csv.reader(fin)
            header = next(reader)
            if first:
                writer.writerow(header)
                first = False
            for row in reader:
                writer.writerow(row)

./ContestData/train2013_all.csv
./ContestData/train2014_all.csv
./ContestData/train2015_all.csv
./ContestData/train2017_all.csv
./ContestData/train2018_all.csv
./ContestData/train2019_all.csv


### Split dataset into training and testing set

In [86]:
file = './ContestData/train_all.csv'
fout_train = './ContestData/train.csv'
fout_test = './ContestData/test.csv'

df = pd.read_csv(file)
# df.drop_duplicates(keep='first', inplace=True)
df = df.sample(frac=1.0)
cut_idx = int(round(0.1 * df.shape[0]))
df_test, df_train = df.iloc[:cut_idx], df.iloc[cut_idx:]
df_train.to_csv(fout_train)
df_test.to_csv(fout_test)
print(df.shape, df_test.shape, df_train.shape)

(1262, 38) (126, 38) (1136, 38)
